In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!tar -xvf /content/20news-bydate.tar.gz

流式输出内容被截断，只能显示最后 5000 行内容。
20news-bydate-train/rec.sport.hockey/53807
20news-bydate-train/rec.sport.hockey/53814
20news-bydate-train/rec.sport.hockey/53815
20news-bydate-train/rec.sport.hockey/53878
20news-bydate-train/rec.sport.hockey/53821
20news-bydate-train/rec.sport.hockey/53811
20news-bydate-train/rec.sport.hockey/53818
20news-bydate-train/rec.sport.hockey/53812
20news-bydate-train/rec.sport.hockey/53813
20news-bydate-train/rec.sport.hockey/53823
20news-bydate-train/rec.sport.hockey/53819
20news-bydate-train/rec.sport.hockey/53817
20news-bydate-train/rec.sport.hockey/54094
20news-bydate-train/rec.sport.hockey/53824
20news-bydate-train/rec.sport.hockey/53891
20news-bydate-train/rec.sport.hockey/53829
20news-bydate-train/rec.sport.hockey/53899
20news-bydate-train/rec.sport.hockey/53830
20news-bydate-train/rec.sport.hockey/53847
20news-bydate-train/rec.sport.hockey/53831
20news-bydate-train/rec.sport.hockey/53833
20news-bydate-train/rec.sport.hockey/53909
20news-bydate-train/rec.spo

In [3]:
import re
import os
from tqdm import tqdm

def divide_text_with_overlap(text, part_size=100, overlap_size=20):
    """
    Divide a text into parts with a specified size and overlap.
    
    :param text: The input text to be divided.
    :param part_size: The size of each part.
    :param overlap_size: The number of overlapping characters between parts.
    :return: A list of text parts.
    """
    parts = []
    position = 0
    while position < len(text):
        section = text[position:position + part_size]
        if len(section) == part_size:
            parts.append(section)
        position += part_size - overlap_size  # Step considering the overlap
    return parts

def retrieve_author_info(content):
    """
    Retrieve the author information from the content using a regular expression.
    
    :param content: The content from which to extract the author information.
    :return: The author's name or "Unknown" if not found.
    """
    author_match = re.search(r'From:(.*?)(?=\w+:|$)', content, re.DOTALL)
    if author_match:
        return author_match.group(1).strip()
    return "Unknown"

# Main function for processing the dataset
def handle_dataset(path_to_dataset, part_size=100):
    """
    Process the dataset by dividing the text files into fragments and extracting metadata.
    
    :param path_to_dataset: The path to the dataset directory.
    :param part_size: The size of each fragment.
    :return: A tuple containing two lists: text fragments and corresponding metadata.
    """
    text_fragments = []
    meta_data = []
    folder_list = os.listdir(path_to_dataset)

    for folder_name in tqdm(folder_list, desc="Processing folders"):
        current_folder = os.path.join(path_to_dataset, folder_name)
        file_list = os.listdir(current_folder)

        for file_name in file_list:
            file_location = os.path.join(current_folder, file_name)
            with open(file_location, 'r', encoding='latin1') as opened_file:
                raw_text = opened_file.read()
                processed_text = re.sub(r'[^\w\s.,!?-]', '', raw_text.replace('\t', ' ').replace('\n', ' '))  # Remove special characters
                processed_text = re.sub(r'\s+', ' ', processed_text)  # Remove extra spaces
                sections = divide_text_with_overlap(processed_text, part_size)
                found_author = retrieve_author_info(raw_text)

                for fragment_id, section in enumerate(sections):
                    metadata_entry = {
                        'category': folder_name,
                        'fragment_id': f'{file_name}-{fragment_id}',
                        'author': found_author
                    }
                    meta_data.append(metadata_entry)
                    text_fragments.append(section)

    return text_fragments, meta_data

dataset_directory = '/content/20news-bydate-test'
fragments, metadata = handle_dataset(dataset_directory)

100%|██████████| 20/20 [00:03<00:00,  6.00it/s]


In [4]:
!pip install -U sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.1 MB/s eta 0:00

In [5]:

from sentence_transformers import SentenceTransformer

class Embedder():
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

    def __call__(self, input):
        input = self.model.encode(input).tolist()
        return input

# embedder = Embedder()



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
import chromadb
import numpy as np
from tqdm import tqdm

class CustomChromaDB:
    def __init__(self):
        """
        Initialize the ChromaDB client with a specified path and embedding model.
        """
        self.embedding_model = Embedder()  # Assuming `Embedder` is a defined class or function for generating embeddings
        self.db_client = chromadb.PersistentClient(path='/content/drive/MyDrive/AT-lab5/ChromaDB')
        self.data_collection = self.db_client.get_or_create_collection(
            name="custom_coll",
            embedding_function=self.embedding_model
        )

    def upload_data(self, fragments, metadata, batch_size=20000):
        """
        Upload data fragments and their corresponding metadata to the ChromaDB collection in batches.

        :param fragments: List of text fragments to be uploaded.
        :param metadata: List of metadata entries corresponding to each fragment.
        :param batch_size: The number of fragments to process in one batch.
        """
        # Generate unique identifiers for the fragments
        fragment_ids = [str(index) for index in range(len(metadata))]
        
        total_batches = (len(fragments) + batch_size - 1) // batch_size  # Calculate the total number of batches
        end = 0
        for batch_idx in tqdm(range(total_batches), desc="Uploading data"):
            start = batch_idx * batch_size
            end = start + batch_size
            batch_fragments = fragments[start:end]
            batch_metadata = metadata[start:end]
            batch_ids = fragment_ids[start:end]

            # Add the batch of fragments to the collection
            self.data_collection.add(
                documents=batch_fragments,
                embeddings=self.embedding_model(batch_fragments),
                metadatas=batch_metadata,
                ids=batch_ids
            )

        # Add any remaining fragments that didn't fit into the last full batch
        remaining_fragments = fragments[end:]
        if remaining_fragments:
            self.data_collection.add(
                documents=remaining_fragments,
                embeddings=self.embedding_model(remaining_fragments),
                metadatas=metadata[end:],
                ids=fragment_ids[end:]
            )

        print("Dataset successfully uploaded to ChromaDB.")

    def search(self, text, count=1):
        """
        Search for the most similar documents to the given text in the ChromaDB collection.

        :param text: The query text to search for.
        :param count: The number of results to return.
        :return: A dictionary containing the search results.
        """
        vector = self.embedding_model(text)
        result = self.data_collection.query(
            query_embeddings=vector,
            n_results=count,
            include=['distances', 'embeddings', 'documents', 'metadatas'],
        )
        return result

In [7]:
cdb = CustomChromaDB()
cdb.upload_data(fragments, metadata)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 7/7 [19:00<00:00, 162.89s/it]


Dataset successfully uploaded to ChromaDB.


In [8]:
cdb = CustomChromaDB()
questions = [
    ['Do you have a copy?', 2],
    ['Have you seen a local news?', 2],
    ['Is this story is true?', 3],
    ['What do you know about computer science?', 4],
    ['Can you give me some information about space?', 5]
]
count = 0
for question in questions:
    result = cdb.search(question[0], question[1])
    print(count)
    print(question[0])
    print(result['documents'])
    print()
    count +=1

0
Do you have a copy?
[[' wanted to know if anyone could send me a copy. I think it should be made into an FAQ, if we can fin', 'd I was wondering if anyone knew where to obtain a copy. I would appreciate any information, as I wo']]

1
Have you seen a local news?
[[' I have been reading the postings in this news group which I just found a few days ago, and I have a', 'past with news of this sort. Does anyone know if this is so or has heard anything of the sort? With ']]

2
Is this story is true?
[['ch in front of my eyes. Are these stories lies? Have the eye-witnesses been day-dreaming? Were these', ' in her stomach in front of my eyes. Are these stories lies? Have the eye-witnesses been day-dreamin', 'cation. The comment by one reporter does make us wonder if this happened but by no means proves it. ']]

3
What do you know about computer science?
[[' more computer knowledge, for a project youre working on.... otherwise, i guess i dont know if id be', 'f Computer Science, University of

In [ ]:
import pinecone
import numpy as np
from tqdm import tqdm

class CustomPinecone():
    def __init__(self, api_key, environment, index_name, embedding_model):
        # Initialize Pinecone client
        pinecone.init(api_key=api_key, environment=environment)

        # Check if the specified index already exists, create it if not
        if index_name not in pinecone.list_indexes():
            pinecone.create_index(index_name, dimension=embedding_model.dimension)

        self.index = pinecone.Index(index_name)
        self.embedding_model = embedding_model  # Assuming Embedder() is a class that can generate embeddings

    def upload_data(self, fragments, metadata, batch_size=100):
        # Associate metadata with fragments and prepare for upload
        items_to_upload = []
        for i, (fragment, meta) in enumerate(zip(fragments, metadata)):
            vector = self.embedding_model(fragment).tolist()
            item = (str(i), vector, meta)
            items_to_upload.append(item)

            # Upload data in batches
            if len(items_to_upload) == batch_size:
                self.index.upsert(vectors=items_to_upload)
                items_to_upload = []

        # Upload remaining data
        if items_to_upload:
            self.index.upsert(vectors=items_to_upload)

        print("Dataset successfully uploaded to Pinecone.")

    def search(self, text, top_k=1):
        # Generate query vector
        query_vector = self.embedding_model(text).tolist()
        # Query the index and return results
        results = self.index.query(queries=[query_vector], top_k=top_k, include_metadata=True)
        return results

In [11]:
cpdb = CustomPinecone()
questions = [
    ['Do you have a copy?'],
    ['Have you seen a local news?'],
    ['Is this story is true?'],
    ['What do you know about computer science?'],
    ['Can you give me some information about space?']
]
count = 0
for question in questions:
    result = cpdb.search(question[0], question[1])
    print(count)
    print(question[0])
    print(result['documents'])
    print()
    count +=1

0
Do you have a copy?
[[' wanted to know if anyone could send me a copy. I think it should be made into an FAQ, if we can fin']]

1
Have you seen a local news?
[[' I have been reading the postings in this news group which I just found a few days ago, and I have a']]

2
Is this story is true?
[['ch in front of my eyes. Are these stories lies? Have the eye-witnesses been day-dreaming? Were these']]

3
What do you know about computer science?
[[' more computer knowledge, for a project youre working on.... otherwise, i guess i dont know if id be']]

4
Can you give me some information about space?
[[' about space. -- Josh Hopkins jbh55289uxa.cso.uiuc.edu Find a way or make one. -attributed to Hannib']]
